In [1]:
pip install kaggle kagglehub


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import kagglehub

# Download the dataset from Kaggle
path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-dataset")

print("Dataset downloaded to:", path)
